In [1]:
import site
import pickle as pkl
import h5py as hp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import glob
import copy

SPECS = {}
SPECS['all'] = ['box', 'simname', 'grid_resolution', 'snapshot', 'axis', 'path', 'mas', 'space']
SPECS['hisubhalo'] = ['model', 'HI_res', 'censat']
SPECS['galaxy'] = ['color', 'gal_species', 'gal_res', 'color_cut', 'censat']
SPECS['ptl'] = ['ptl_species']
SPECS['hiptl'] = ['model']
SPECS['vn'] = []

def siteFG():
    FGPATH = '/home/cosinga/figrid/'
    site.addsitedir(FGPATH)
    return

def loadpks(dl):
    path = '/lustre/cosinga/hcolor/output/*/results/*.pkl_rlib.pkl'
    filenames = glob.glob(path)
    total = 0
    for f in range(len(filenames)):
        fl = pkl.load(open(filenames[f], 'rb'))
        newprops = {'path':filenames[f].split('/')[5].split('_')[0]}
        if 'pk' in fl.results:
            total += len(fl.results['pk'])
            dl.loadResults(fl.results['pk'], newprops)
        print("%.2f"%(f/len(filenames)*100) + r"% loaded")
    
    return dl

def makeRSD(datalist):
    from figrid.data_container import DataContainer
    ip = {'space':'real', 'path':'fiducial', 'post_process':'no key found'}
    real = datalist.getMatching(ip)
    rsdlist = []
    for dc in real:
        mattr = copy.deepcopy(dc.attrs)
        rmattr = []
        for k in mattr:
            if 'runtime' in k or 'space' in k:
                rmattr.append(k)
        for rm in rmattr:
            del mattr[rm]
        mattr['space'] = 'redshift'
        redshift = datalist.getMatching(mattr)
        if len(redshift) > 1:
            print('%d corresponding redshift-space for a real-space:'%len(redshift))
            print("real ATTRS:")
            print(dc.attrs)
            print("redshift ATTRS:")
            for r in redshift:
                print(r.attrs)
            
        elif len(redshift) == 0:
            print('no redshift space counterparts found')
            print(dc.attrs)
            continue
        else:
            redshift = redshift[0]
            data = [dc.data[0], redshift.data[1]/dc.data[1]]
            rsd = DataContainer(data)
            mattr['space'] = 'rsd' 
            rsd.update(mattr)
            rsdlist.append(rsd)
    return rsdlist

def makeBlueRedRatio(datalist):
    from figrid.data_container import DataContainer
    ip = {'color':'blue', 'color_cut':'0.60', 'path':'fiducial', 'post_process':'no key found'}
    blues = datalist.getMatching(ip)
    ratiolist = []
    for dc in blues:
        mattr = copy.deepcopy(dc.attrs)
        rmattr = []
        for k in mattr:
            if 'runtime' in k or 'color' == k:
                rmattr.append(k)
        for rm in rmattr:
            del mattr[rm]
        mattr['color'] = 'red'
        reds = datalist.getMatching(mattr)
        if len(reds) > 1:
            print('%d corresponding reds for a blue:'%len(reds))
            print("BLUE ATTRS:")
            print(dc.attrs)
            print("RED ATTRS:")
            for r in reds:
                print(r.attrs)
        if len(reds) == 0:
            print('no reds found')
            print(dc.attrs)
        else:
            reds = reds[0]
            data = [dc.data[0], reds.data[1]/dc.data[1]]
            ratio = DataContainer(data)
            mattr['color'] = 'ratio' 
            ratio.update(mattr)
            ratiolist.append(ratio)
    return ratiolist

def makeBias(datalist):
    from figrid.data_container import DataContainer
    biaslist = []
    
    crosses = datalist.getMatching({'is_auto' : False, 'path':['fiducial']})
    for cx in crosses:
        fns = cx.get('fieldname').split('_')
        if 'dust' in fns:
            continue
        autos = []
        for f in fns:
            attrlist = copy.deepcopy(SPECS['all'])
            attrlist.extend(SPECS[f])
            attr = {'is_auto':True, 'fieldname':f}
            for at in attrlist:
                attr[at] = cx.get(at)
            matches = datalist.getMatching(attr)
            if len(matches)>1:
                print('too many matches %d'%len(matches))
                for m in matches:
                    print(m.attrs)
            elif len(matches) == 0:
                print('no matches found')
                print(attr)
                print(cx.attrs)
            else:
                auto = matches[0]
                data = [cx.data[0], cx.data[1] / auto.data[1]]
                dc = DataContainer(data)
                dc.update(copy.deepcopy(cx.attrs))
                dc.add('post_process', 'theory_bias')
                dc.add('numerator', cx.get('fieldname'))
                dc.add('denominator', auto.get('fieldname'))
                biaslist.append(dc)
                autos.append(auto)
        if len(autos) == 2:
            for a in range(len(autos)):
                data = [autos[a].data[0], np.sqrt(autos[a].data[1] / autos[(a+1)%2].data[1])]
                dc = DataContainer(data)
                dc.update(copy.deepcopy(cx.attrs))
                dc.add('post_process', 'obs_bias')
                dc.add('numerator', autos[a].get('fieldname'))
                dc.add('denominator', autos[(a+1)%2].get('fieldname'))
                biaslist.append(dc)

            data = [autos[0].data[0], cx.data[1] / np.sqrt(autos[0].data[1] * autos[1].data[1])]
            dc = DataContainer(data)
            dc.update(copy.deepcopy(cx.attrs))
            dc.add('post_process', 'corr_coef')
            biaslist.append(dc)
            
        del attr
    return biaslist

siteFG()
from figrid.data_sort import DataSort
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rcParams['font.family'] = 'serif'

In [2]:
def resaveSort(name):
    ds = DataSort()
    ds = loadpks(ds)
    ds.extend(makeBlueRedRatio(ds))
    ds.extend(makeRSD(ds))
    pkl.dump(ds, open(name, 'wb'), pkl.HIGHEST_PROTOCOL)
    return
resaveSort('/lustre/cosinga/hcolor/fig_md_analysis/6-1_datasort.pkl')

0.00% loaded
0.56% loaded
1.13% loaded
1.69% loaded
2.26% loaded
2.82% loaded
3.39% loaded
3.95% loaded
4.52% loaded
5.08% loaded
5.65% loaded
6.21% loaded
6.78% loaded
7.34% loaded
7.91% loaded
8.47% loaded
9.04% loaded
9.60% loaded
10.17% loaded
10.73% loaded
11.30% loaded
11.86% loaded
12.43% loaded
12.99% loaded
13.56% loaded
14.12% loaded
14.69% loaded
15.25% loaded
15.82% loaded
16.38% loaded
16.95% loaded
17.51% loaded
18.08% loaded
18.64% loaded
19.21% loaded
19.77% loaded
20.34% loaded
20.90% loaded
21.47% loaded
22.03% loaded
22.60% loaded
23.16% loaded
23.73% loaded
24.29% loaded
24.86% loaded
25.42% loaded
25.99% loaded
26.55% loaded
27.12% loaded
27.68% loaded
28.25% loaded
28.81% loaded
29.38% loaded
29.94% loaded
30.51% loaded
31.07% loaded
31.64% loaded
32.20% loaded
32.77% loaded
33.33% loaded
33.90% loaded
34.46% loaded
35.03% loaded
35.59% loaded
36.16% loaded
36.72% loaded
37.29% loaded
37.85% loaded
38.42% loaded
38.98% loaded
39.55% loaded
40.11% loaded
40.68% loa